# AWS Cost and Usage Report Analysis

The report type being analyzed here is the Detailed Hourly AWS Cost and Usage Report with Resource IDs and Tags. These reports are downloaded using the [get-aws-cost.py](https://github.com/OPENDAP/cloudydap/blob/master/python/logging/get-aws-cost.py) command-line program developed for this project.

In [1]:
import numpy as np
import pandas as pd
from aws_price_list import AWSOffersIndex
from bokeh.charts import (output_notebook, output_file, show, 
                          Scatter, Histogram, TimeSeries, Donut, Step)
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import Range1d, HoverTool, ResizeTool


output_notebook()

Loading BokehJS ...

## Report to Analyze

In [2]:
r = pd.read_csv('Arch1-20170101-20170201-1.csv')

/Users/ajelenak/Documents/dmr++/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (47,48,57) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


What do we have?

In [3]:
r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15814 entries, 0 to 15813
Data columns (total 76 columns):
identity/LineItemId                    15814 non-null object
identity/TimeInterval                  15814 non-null object
bill/InvoiceId                         15814 non-null int64
bill/BillingEntity                     15814 non-null object
bill/BillType                          15814 non-null object
bill/PayerAccountId                    15814 non-null int64
bill/BillingPeriodStartDate            15814 non-null object
bill/BillingPeriodEndDate              15814 non-null object
lineItem/UsageAccountId                15814 non-null int64
lineItem/LineItemType                  15814 non-null object
lineItem/UsageStartDate                15814 non-null object
lineItem/UsageEndDate                  15814 non-null object
lineItem/ProductCode                   15814 non-null object
lineItem/UsageType                     15814 non-null object
lineItem/Operation                     1

Convert two columns to datetime, and sort all entries by time:

In [4]:
r['lineItem/UsageStartDate'] = pd.to_datetime(r['lineItem/UsageStartDate'])
r['lineItem/UsageEndDate'] = pd.to_datetime(r['lineItem/UsageEndDate'])
r.sort_values('lineItem/UsageStartDate', inplace=True)

In [5]:
r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15814 entries, 0 to 15813
Data columns (total 76 columns):
identity/LineItemId                    15814 non-null object
identity/TimeInterval                  15814 non-null object
bill/InvoiceId                         15814 non-null int64
bill/BillingEntity                     15814 non-null object
bill/BillType                          15814 non-null object
bill/PayerAccountId                    15814 non-null int64
bill/BillingPeriodStartDate            15814 non-null object
bill/BillingPeriodEndDate              15814 non-null object
lineItem/UsageAccountId                15814 non-null int64
lineItem/LineItemType                  15814 non-null object
lineItem/UsageStartDate                15814 non-null datetime64[ns]
lineItem/UsageEndDate                  15814 non-null datetime64[ns]
lineItem/ProductCode                   15814 non-null object
lineItem/UsageType                     15814 non-null object
lineItem/Operation      

## AWS Price Information

In [6]:
oi = AWSOffersIndex()

In [7]:
oi.published.strftime('%c %Z')

'Fri Feb 10 22:31:44 2017 UTC+00:00'

In [8]:
oi.accessed.strftime('%c %Z')

'Mon Feb 20 04:57:11 2017 UTC+00:00'

## Analysis

Time span of the report's data:

In [9]:
r['lineItem/UsageStartDate'].min()

Timestamp('2017-01-01 00:00:00')

In [10]:
r['lineItem/UsageEndDate'].max()

Timestamp('2017-02-01 00:00:00')

How many different Availability Zones?

In [11]:
r['lineItem/AvailabilityZone'].unique()

array(['us-east-1b', nan], dtype=object)

How many different product codes?

In [12]:
r['lineItem/ProductCode'].unique()

array(['AmazonEC2', 'AmazonS3', 'awskms', 'AmazonSNS', 'AWSQueueService'], dtype=object)

How many report entries for each product code?

In [13]:
r['lineItem/ProductCode'].value_counts()

AmazonEC2          10418
AmazonS3            5390
awskms                 2
AmazonSNS              2
AWSQueueService        2
Name: lineItem/ProductCode, dtype: int64

Cost breakdown per AWS product:

In [14]:
grp = r.groupby('lineItem/ProductCode')
x = grp['lineItem/BlendedCost'].sum()
x

lineItem/ProductCode
AWSQueueService      0.000000
AmazonEC2          455.481245
AmazonS3            26.723635
AmazonSNS            0.000000
awskms               0.000000
Name: lineItem/BlendedCost, dtype: float64

In [15]:
f = Donut(x.iloc[x.nonzero()], title='AWS Product Cost Breakdown')
show(f)

### Analysis: AmazonEC2

In [16]:
ec2 = r[r['lineItem/ProductCode'] == 'AmazonEC2']

How many usage types?

In [17]:
ec2['lineItem/UsageType'].value_counts()

EBS:VolumeUsage.gp2            2232
DataTransfer-In-Bytes          1994
BoxUsage:m4.xlarge             1994
EBSOptimized:m4.xlarge         1994
DataTransfer-Out-Bytes         1994
DataTransfer-Regional-Bytes      90
EBS:SnapshotUsage                32
USE1-USW2-AWS-In-Bytes           13
USE1-USW2-AWS-Out-Bytes          13
USE1-APN2-AWS-Out-Bytes           8
USE1-EU-AWS-In-Bytes              8
USE1-EU-AWS-Out-Bytes             8
USE1-APN2-AWS-In-Bytes            8
USE1-APS1-AWS-In-Bytes            6
USE1-APS1-AWS-Out-Bytes           6
USE1-USW1-AWS-Out-Bytes           4
USE1-USW1-AWS-In-Bytes            4
USE1-APN1-AWS-Out-Bytes           3
USE1-APN1-AWS-In-Bytes            3
USE1-SAE1-AWS-Out-Bytes           2
USE1-SAE1-AWS-In-Bytes            2
Name: lineItem/UsageType, dtype: int64

Cost breakdown for different product types:

In [18]:
grp = ec2.groupby('lineItem/UsageType')

In [19]:
x = grp['lineItem/BlendedCost'].sum()
x

lineItem/UsageType
BoxUsage:m4.xlarge             4.287100e+02
DataTransfer-In-Bytes          0.000000e+00
DataTransfer-Out-Bytes         0.000000e+00
DataTransfer-Regional-Bytes    0.000000e+00
EBS:SnapshotUsage              1.771243e+00
EBS:VolumeUsage.gp2            2.500000e+01
EBSOptimized:m4.xlarge         0.000000e+00
USE1-APN1-AWS-In-Bytes         0.000000e+00
USE1-APN1-AWS-Out-Bytes        0.000000e+00
USE1-APN2-AWS-In-Bytes         0.000000e+00
USE1-APN2-AWS-Out-Bytes        0.000000e+00
USE1-APS1-AWS-In-Bytes         0.000000e+00
USE1-APS1-AWS-Out-Bytes        0.000000e+00
USE1-EU-AWS-In-Bytes           0.000000e+00
USE1-EU-AWS-Out-Bytes          0.000000e+00
USE1-SAE1-AWS-In-Bytes         0.000000e+00
USE1-SAE1-AWS-Out-Bytes        0.000000e+00
USE1-USW1-AWS-In-Bytes         0.000000e+00
USE1-USW1-AWS-Out-Bytes        0.000000e+00
USE1-USW2-AWS-In-Bytes         0.000000e+00
USE1-USW2-AWS-Out-Bytes        9.200000e-07
Name: lineItem/BlendedCost, dtype: float64

In [20]:
f = Donut(x.iloc[x.nonzero()], title='AWS EC2 Product Cost Breakdown',
          plot_height=600, plot_width=600)
show(f)

How many unique operations for each of the AmazonEC2 product types?

In [21]:
grp['lineItem/Operation'].unique()

lineItem/UsageType
BoxUsage:m4.xlarge                          [RunInstances]
DataTransfer-In-Bytes                       [RunInstances]
DataTransfer-Out-Bytes                      [RunInstances]
DataTransfer-Regional-Bytes    [PublicIP-Out, PublicIP-In]
EBS:SnapshotUsage                         [CreateSnapshot]
EBS:VolumeUsage.gp2                     [CreateVolume-Gp2]
EBSOptimized:m4.xlarge                            [Hourly]
USE1-APN1-AWS-In-Bytes                       [PublicIP-In]
USE1-APN1-AWS-Out-Bytes                     [PublicIP-Out]
USE1-APN2-AWS-In-Bytes                       [PublicIP-In]
USE1-APN2-AWS-Out-Bytes                     [PublicIP-Out]
USE1-APS1-AWS-In-Bytes                       [PublicIP-In]
USE1-APS1-AWS-Out-Bytes                     [PublicIP-Out]
USE1-EU-AWS-In-Bytes                         [PublicIP-In]
USE1-EU-AWS-Out-Bytes                       [PublicIP-Out]
USE1-SAE1-AWS-In-Bytes                       [PublicIP-In]
USE1-SAE1-AWS-Out-Bytes              

How many unique product SKUs for each of the AmazonEC2 product types?

In [22]:
skus = grp['product/sku'].unique()
skus

lineItem/UsageType
BoxUsage:m4.xlarge             [47GP959QAF69YPG5]
DataTransfer-In-Bytes          [9MG5B7V4UUU2WPAV]
DataTransfer-Out-Bytes         [HQEH3ZWJVT46JHRG]
DataTransfer-Regional-Bytes    [PNUBVW4CPC8XA46W]
EBS:SnapshotUsage              [7U7TWP44UP36AT3R]
EBS:VolumeUsage.gp2            [HY3BZPP2B6K8MSJF]
EBSOptimized:m4.xlarge         [9W95WEA2F9V4BVUJ]
USE1-APN1-AWS-In-Bytes         [BJKH3VYGVEEXSNAH]
USE1-APN1-AWS-Out-Bytes        [EDJPFY8K3AW2AD23]
USE1-APN2-AWS-In-Bytes         [C7XDWJNQJCHH2ZQT]
USE1-APN2-AWS-Out-Bytes        [Y8HYRU3BZZ4QFJHH]
USE1-APS1-AWS-In-Bytes         [DZHU5BKVVZXEHEYR]
USE1-APS1-AWS-Out-Bytes        [2T92AZQGNFAQHEXW]
USE1-EU-AWS-In-Bytes           [725FHGTUB3P2B9EU]
USE1-EU-AWS-Out-Bytes          [NW4B786HNAH6HZ7R]
USE1-SAE1-AWS-In-Bytes         [R8DGVR4A52UQ7VP6]
USE1-SAE1-AWS-Out-Bytes        [X67HWFH8QVT6NSW7]
USE1-USW1-AWS-In-Bytes         [NBUQPTSYHSXS2EB6]
USE1-USW1-AWS-Out-Bytes        [8X3QU4DYXVJAXZK3]
USE1-USW2-AWS-In-Bytes         

Display description for all of these SKUs and their pricing tiers:

In [23]:
s3o = oi.offer('AmazonEC2')

In [24]:
for ut, sku_list in skus.iteritems():
    print('Usage Type:', ut)
    for sku in sku_list:
        print('  SKU:', sku)
        prod = s3o.product(sku)
        if len(prod.pricing) > 1:
            raise ValueError('{}: More than one product pricing info')
        tiers = prod.pricing[0].tiers
        for t in tiers:
            print('    ', t.description)
        print('\n')

Usage Type: BoxUsage:m4.xlarge
  SKU: 47GP959QAF69YPG5
     $0.215 per On Demand Linux m4.xlarge Instance Hour


Usage Type: DataTransfer-In-Bytes
  SKU: 9MG5B7V4UUU2WPAV
     $0.000 per GB - data transfer in per month


Usage Type: DataTransfer-Out-Bytes
  SKU: HQEH3ZWJVT46JHRG
     $0.050 per GB - greater than 150 TB / month data transfer out
     $0.085 per GB - next 40 TB / month data transfer out
     $0.090 per GB - first 10 TB / month data transfer out beyond the global free tier
     $0.000 per GB - first 1 GB of data transferred out per month
     $0.070 per GB - next 100 TB / month data transfer out


Usage Type: DataTransfer-Regional-Bytes
  SKU: PNUBVW4CPC8XA46W
     $0.010 per GB - regional data transfer - in/out/between EC2 AZs or using elastic IPs or ELB


Usage Type: EBS:SnapshotUsage
  SKU: 7U7TWP44UP36AT3R
     $0.05 per GB-Month of snapshot data stored - US East (Northern Virginia)


Usage Type: EBS:VolumeUsage.gp2
  SKU: HY3BZPP2B6K8MSJF
     $0.10 per GB-month of G

### Analysis: AmazonS3

In [25]:
s3 = r[r['lineItem/ProductCode'] == 'AmazonS3']

How many usage types?

In [26]:
s3['lineItem/UsageType'].value_counts()

USE1-USW2-AWS-Out-Bytes    2231
Requests-Tier2             1720
Requests-Tier1             1109
DataTransfer-Out-Bytes      200
DataTransfer-In-Bytes        98
TimedStorage-ByteHrs         32
Name: lineItem/UsageType, dtype: int64

Cost breakdown for different product types:

In [27]:
grp = s3.groupby('lineItem/UsageType')

In [28]:
x = grp['lineItem/BlendedCost'].sum()
x

lineItem/UsageType
DataTransfer-In-Bytes       0.000000
DataTransfer-Out-Bytes      0.000000
Requests-Tier1             16.781100
Requests-Tier2              0.259584
TimedStorage-ByteHrs        8.002707
USE1-USW2-AWS-Out-Bytes     1.680243
Name: lineItem/BlendedCost, dtype: float64

In [29]:
f = Donut(x.iloc[x.nonzero()], title='AWS S3 Cost Breakdown',
          plot_height=600, plot_width=600)
show(f)

How many unique operations for each of the AmazonS3 product types?

In [30]:
grp['lineItem/Operation'].unique()

lineItem/UsageType
DataTransfer-In-Bytes                                            [PutObject]
DataTransfer-Out-Bytes     [GetObject, ListBucket, ReadVersioningProps, R...
Requests-Tier1             [ListBucketVersions, PutObject, ListBucket, Li...
Requests-Tier2             [HeadBucket, GetObject, ReadACL, HeadObject, R...
TimedStorage-ByteHrs                                       [StandardStorage]
USE1-USW2-AWS-Out-Bytes          [ListBucketVersions, HeadBucket, GetObject]
Name: lineItem/Operation, dtype: object

How many unique product SKUs for each of the AmazonS3 product types?

In [31]:
skus = grp['product/sku'].unique()
skus

lineItem/UsageType
DataTransfer-In-Bytes      [9MG5B7V4UUU2WPAV]
DataTransfer-Out-Bytes     [HQEH3ZWJVT46JHRG]
Requests-Tier1             [E9YHNFENF4XQBZR6]
Requests-Tier2             [ZWQ6Q48CRJXX4FXE]
TimedStorage-ByteHrs       [WP9ANXZGBYYSGJEA]
USE1-USW2-AWS-Out-Bytes    [XGXYRYWGNXSSEUVT]
Name: product/sku, dtype: object

Display description for all of these SKUs and their pricing tiers:

In [32]:
s3o = oi.offer('AmazonS3')

In [33]:
for ut, sku_list in skus.iteritems():
    print('Usage Type:', ut)
    for sku in sku_list:
        print('  SKU:', sku)
        prod = s3o.product(sku)
        if len(prod.pricing) > 1:
            raise ValueError('{}: More than one product pricing info')
        tiers = prod.pricing[0].tiers
        for t in tiers:
            print('    ', t.description)
        print('\n')

Usage Type: DataTransfer-In-Bytes
  SKU: 9MG5B7V4UUU2WPAV
     $0.000 per GB - data transfer in per month


Usage Type: DataTransfer-Out-Bytes
  SKU: HQEH3ZWJVT46JHRG
     $0.050 per GB - greater than 150 TB / month data transfer out
     $0.085 per GB - next 40 TB / month data transfer out
     $0.090 per GB - first 10 TB / month data transfer out beyond the global free tier
     $0.000 per GB - first 1 GB of data transferred out per month
     $0.070 per GB - next 100 TB / month data transfer out


Usage Type: Requests-Tier1
  SKU: E9YHNFENF4XQBZR6
     $0.005 per 1,000 PUT, COPY, POST, or LIST requests


Usage Type: Requests-Tier2
  SKU: ZWQ6Q48CRJXX4FXE
     $0.004 per 10,000 GET and all other requests


Usage Type: TimedStorage-ByteHrs
  SKU: WP9ANXZGBYYSGJEA
     $0.021 per GB - storage used / month over 500 TB
     $0.022 per GB - next 450 TB / month of storage used
     $0.023 per GB - first 50 TB / month of storage used


Usage Type: USE1-USW2-AWS-Out-Bytes
  SKU: XGXYRYWGNXSS

## Sandbox

Toying with various ideas below... Eventually, this stuff may be pulled somewhere up.

In [41]:
f = Step(s3[(s3['lineItem/UsageType'] == 'TimedStorage-ByteHrs') & 
            (s3['lineItem/Operation'] == 'StandardStorage')],
         x='lineItem/UsageStartDate', y='lineItem/UsageAmount', xscale='datetime')
show(f)